# Week 9 Assignment: Encrypted QAP

src: <https://almondine-song-c43.notion.site/Homework-8-Encrypted-QAP-4d75259eac0246aa86ed48b51e7700b6>

Given an R1CS, you should transform it into a QAP (you can use the code from the RareSkills ZK Book for this).

This will produce polynomials U, V, W, and HT (see the notation in the book).

Do an encrypted evaluation of each of these polynomials, this will result in

$$
\begin{matrix}
eval(U) = [A]_1\\eval(V)=[B]_2,\\ eval(W)=[C']_1,\\ eval(HT)=[HT]_1
\end{matrix}
$$

Create

$$
[C]=[C']_1+[HT]_1
$$

Then verify that

$$
\text{pairing}([A]_1,[B]_2)-\text{pairing}([C]_1,[G]_2) = 0
$$

Do the verification on chain.

Your code should be able to start with an arbitrary R1CS and compute the three elliptic curve points

$$
[A]_1, [B]_2, [C]_1
$$

for which the verifier will check if the pairing is correct.

In [27]:
import numpy as np
from scipy.interpolate import lagrange
from functools import reduce
import galois

def to_galois(n: int, GF: galois.FieldArray) -> galois:
    positive = n
    while positive < 0: positive += GF.order
    return GF(positive % GF.order)

def r1cs_to_qap(L: np.array, R: np.array, O: np.array, w: np.array, GF: galois.FieldArray) -> (np.array, np.array, np.array, np.array, np.array):
    """
    Convert an R1CS to QAP. All the computation is done in finite field
    They need to satisfy this relationship: L·w ⊙ R·w = O·w

    Paramters:
    L (np.array): m x n array, wrapped in Galois field
    R (np.array): m x n array, wrapped in Galois field
    O (np.array): m x n array, wrapped in Galois field
    w (np.array): n x 1 vector, wrapped in Galois field
    GF (galois.FieldArray): The finite field order

    Returns:

    Raises:
    """
    # Check for the params size
    if (L.ndim != 2 or R.ndim != 2 or O.ndim != 2 or w.ndim != 1
        or L.shape[0] != R.shape[0] or R.shape[0] != O.shape[0]
        or L.shape[1] != R.shape[1] or R.shape[1] != O.shape[1]
        or w.shape[0] != O.shape[1]
       ):
        raise Exception("Parameters sizes mismatched.")
    
    def interpolate_col(col):
        xs = GF(np.array(range(1, len(col) + 1)))
        return galois.lagrange_poly(xs, col)

    def inner_product_polys_with_w(polys, witness):
        mul_ = lambda x, y: x * y
        sum_ = lambda x, y: x + y
        return reduce(sum_, map(mul_, polys, witness))

    def get_t_poly(poly: galois.Poly) -> galois.Poly:
        """
        The return value is a polynomial with deg(poly) - 1 
        """
        return galois.Poly.Roots(list(range(1, poly.degree)), field = GF)
        

    Lg = GF(L)
    Rg = GF(R)
    Og = GF(O)

    U = np.apply_along_axis(interpolate_col, 0, Lg)
    V = np.apply_along_axis(interpolate_col, 0, Rg)
    W = np.apply_along_axis(interpolate_col, 0, Og)

    # Uw·Vw = Ww + h(x)t(x)
    Uw = inner_product_polys_with_w(U, w)
    Vw = inner_product_polys_with_w(V, w)
    Ww = inner_product_polys_with_w(W, w)

    # Uw·Vw - Ww
    result_poly = Uw * Vw - Ww
    T = get_t_poly(result_poly)
    H = result_poly // T

    return (Uw, Vw, Ww, H, T)

def main():
    # Define the Galois field
    GF = galois.GF(79)    
    
    # Define the matrices
    L = np.array([[0,0,3,0,0,0],
                  [0,0,0,0,1,0],
                  [0,0,1,0,0,0]])

    R = np.array([[0,0,1,0,0,0],
                  [0,0,0,1,0,0],
                  [0,0,0,5,0,0]])

    O = np.array([[0,0,0,0,1,0],
                  [0,0,0,0,0,1],
                  [-3,1,1,2,0,-1]])

    # witness
    x = to_galois(100, GF)
    y = to_galois(100, GF)
    v1 = to_galois(3, GF) * x * x
    v2 = v1 * y
    out = v2 + to_galois(5, GF) * x * y - x - to_galois(2, GF) * y + to_galois(3, GF)
    witness = GF(np.array([1, out, x, y, v1, v2]))

    # Convert ndarray into Galois field
    Lg = GF(np.array([[to_galois(x, GF) for x in row] for row in L]))
    Rg = GF(np.array([[to_galois(x, GF) for x in row] for row in R]))
    Og = GF(np.array([[to_galois(x, GF) for x in row] for row in O]))

    ret_values = r1cs_to_qap(Lg, Rg, Og, witness, GF)
    print(ret_values)

main()

(Poly(62x^2 + 47x + 33, GF(79)), Poly(42x^2 + 32x + 26, GF(79)), Poly(51x^2 + 8, GF(79)), Poly(76x + 69, GF(79)), Poly(x^3 + 73x^2 + 11x + 73, GF(79)))
